<a href="https://colab.research.google.com/github/vegarab/msc-qg/blob/master/notebooks/T5_question_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 757kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 14.1MB/s 
     |████████████████████████████████| 3.0MB 21.5MB/s 
     |████████████████████████████████| 890kB 46.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b04008a9d15f3d5312bed6960ef9d7d77343724a536061226dc953cbaaa179fc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch

from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer
)

In [4]:
!nvidia_smi

/bin/bash: nvidia_smi: command not found


In [47]:
class T5QuestionGenerator():
    def __init__(self, max_len=512, q_len=64):
        model_file = './drive/My Drive/msc-qg/models/t5_squad_exploration_2/'
        self.model = T5ForConditionalGeneration.from_pretrained(model_file)
        self.tokenizer = T5Tokenizer.from_pretrained('t5-base')

        self.model.eval()

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)

        self.MAX_LEN = max_len
        self.Q_LEN = q_len

    def generate(self, context, answer):
        source = self._encode_input(context, answer)

        source_ids = source['input_ids'].cuda().to(dtype=torch.long)
        source_mask = source['attention_mask'].cuda().to(dtype=torch.long)
        
        generated_ids = self.model.generate(
            input_ids=source_ids,
            attention_mask=source_mask,
            max_length=self.MAX_LEN,
            num_beams=2,
            repition_penalty=2.5,
            early_stopping=True
        )

        question = self._decode_question(generated_ids)

        return 'question' + question

    def _encode_input(self, context, answer):
        context = 'context: ' + context
        answer = 'answer: ' + answer
        _input = answer + ' ' + context + ' </s>'


        return self.tokenizer.batch_encode_plus([_input],
                                                max_length=self.MAX_LEN,
                                                pad_to_max_length=True,
                                                truncation=True,
                                                return_tensors='pt')

    def _decode_question(self, generated_ids):
        # Assuming only one input -> one output
        generated_id = generated_ids.squeeze()
        predicted = self.tokenizer.decode(generated_id, 
                                          skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
        return predicted


In [48]:
generator = T5QuestionGenerator()

In [69]:
# https://www.inspera.com/about
context = """Inspera Assessment was created to provide test takers with equal opportunities 
             and to enable them to prove their skills in an authentic way. We empower universities, 
             awarding organisations, governments, municipalities and schools to deliver secure, 
             standardised tests, open- and closed- book exams, as well as formative and diagnostics 
             tests. Inspera offers the most innovative, reliable and secure online assessment platform, 
             accompanied with the necessary expertise to ensure successful organisation-wide 
             implementation and rollout. Founded in 1999, we are a dedicated international team of more 
             than 90 EdTech entrepreneurs on a mission to impact education on a global scale. """

answers = ['1999', 'an online assessment platform', 'governments', 
           'international team', 'an international team', 'global']

for answer in answers:
    print('answer: ' + answer)
    print(generator.generate(context, answer) + '\n')

answer: 1999
question: When was Inspera Assessment founded?

answer: an online assessment platform
question: What type of platform does Inspera offer?

answer: governments
question: Along with universities and awarding organisations, what other organization does Inspera Assessment work with?

answer: international team
question: What type of team is Inspera?

answer: an international team
question: What is Inspera Assessment's mission?

answer: global
question: On what scale is Inspera Assessment a dedicated team of 90 EdTech entrepreneurs?



In [70]:
# https://www.ielts.org/-/media/pdfs/pb-sample-test-materials/academic-reading-sample-task-multiple-choice-one-answer.ashx?la=en
context = """The general assumption is that older workers are paid more in spite of, rather 
             than because of, their productivity. That might partly explain why, when employers 
             are under pressure to cut costs, they persuade a 55-year old to take early retirement. 
             Take away seniority-based pay scales, and older workers may become a much more 
             attractive employment proposition. But most employers and many workers are uncomfortable 
             with the idea of reducing someone’s pay in later life –although manual workers on piece-rates 
             often earn less as they get older. So retaining the services of older workers may mean 
             employing them in different ways.
             
             One innovation was devised by IBM Belgium. Faced with the need to cut staff costs, and having 
             decided to concentrate cuts on 55 to 60-year olds, IBM set up a separate company called Skill 
             Team, which re-employed any of the early retired who wanted to go on working up to the age of 
             60. An employee who joined Skill Team at the age of 55 on a five-year contract would work for 
             58% of his time, over the full period, for 88% of his last IBM salary. The company offered 
             services to IBM, thus allowing it to retain access to some of the intellectual capital it 
             would otherwise have lost.
             
             The best way to tempt the old to go on working may be to build on such ‘bridge’ jobs: part-time 
             or temporary employment that creates a more gradual transition from full-time work to retirement. 
             Studies have found that, in the United States, nearly half of all men and women who had been in 
             full-time jobs in middle age moved into such ‘bridge’ jobs at the end of their working lives. 
             In general, it is the best-paid and worst-paid who carry on working. There seem to be two very 
             different types of bridge job-holder –those who continue working because they have to and those 
             who continue working because they want to, even though theycould afford to retire.
             
             If the job market grows more flexible, the old may find more jobs that suit them. 
             Often, they will be self-employed. Sometimes, they may start their own businesses: a study by 
             David Storey of Warwick University found that in Britain 70% of businesses started by people over 
             55 survived, compared with an overall national average of only 19%. But whatever pattern of 
             employment they choose, in the coming years the skills of these ‘grey workers’ will have to be 
             increasingly acknowledgedand rewarded."""

answers = ['abolishing pay schemes that are based on age',
           'allows the expertise of older workers to be put to use',
           'appeal to distinct groups of older workers',
           'older people are good at running their own businesses']

true_questions = ['The writer suggests that companies could consider...',
                  'Skill Team is an example of a company which...',
                  'According to the writer, "bridge" jobs...',
                  'David Storey\'s study found that...']

for i,answer in enumerate(answers):
    print('answer: ' + answer)
    print(generator.generate(context, answer))
    print('true question: ' + true_questions[i] + '\n')

answer: abolishing pay schemes that are based on age
question: What is the best way to tempt the old to work?
true question: The writer suggests that companies could consider...

answer: allows the expertise of older workers to be put to use
question: What does the Skill Team do?
true question: Skill Team is an example of a company which...

answer: appeal to distinct groups of older workers
question: What may mean employing older workers in different ways?
true question: According to the writer, "bridge" jobs...

answer: older people are good at running their own businesses
question: What is the general assumption about older workers?
true question: David Storey's study found that...

